### Today we are going to perform the simple classification of the amazon reviews' sentiment.

### Please, download the dataset amazon_baby.csv.

In [218]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.linear_model import LogisticRegression

def remove_punctuation(text):
    import string
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

baby_df = pd.read_csv('amazon_baby.csv')
baby_df.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


## Exercise 1 (data preparation)
a) Remove punctuation from reviews using the given function.   
b) Replace all missing (nan) revies with empty "" string.  
c) Drop all the entries with rating = 3, as they have neutral sentiment.   
d) Set all positive ($\geq$4) ratings to 1 and negative($\leq$2) to -1.

In [219]:
#a) 
#At the beggining we have to remove punctuation and fill empty cells in database to prepare data for classification. We want to avoid situations when eg.
#words 'love' and 'love!' are understood by model as two different words.
baby_df['review'] = baby_df['review'].astype(str).apply(remove_punctuation)
#short test:
baby_df["review"][4] == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'
remove_punctuation(baby_df["review"][4]) == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'

True

In [220]:
#b)
baby_df['review'] = baby_df['review'].fillna("")
#short test:
baby_df["review"][38] == baby_df["review"][38]

True

In [221]:
#c)
#Then we are removing reviews where rating has a middle value and changing values for low rating for 1 and high rating for -1. 
#It's usually done in classification model which should focus more on extreme values.
baby_df = baby_df[baby_df["rating"] != 3]
print(baby_df["rating"].value_counts())

#short test:
sum(baby_df["rating"] == 3)

rating
5    107054
4     33205
1     15183
2     11310
Name: count, dtype: int64


0

In [222]:
#d)
baby_df.loc[baby_df["rating"] <= 2, "rating"] = -1
baby_df.loc[baby_df["rating"] >= 4, "rating"] = 1
print(baby_df["rating"].value_counts())

#short test:
sum(baby_df["rating"]**2 != 1)

rating
 1    140259
-1     26493
Name: count, dtype: int64


0

## CountVectorizer
In order to analyze strings, we need to assign them numerical values. We will use one of the simplest string representation, which transforms strings into the $n$ dimensional vectors. The number of dimensions will be the size of our dictionary, and then the values of the vector will represent the number of appereances of the given word in the sentence.

In [223]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
reviews_train_example = ["We like apples",
                   "We hate oranges",
                   "I adore bananas",
                   "We like like apples and oranges",
                   "They dislike bananas"]

X_train_example = vectorizer.fit_transform(reviews_train_example)

print(vectorizer.get_feature_names_out())
print(X_train_example.todense())



['adore' 'and' 'apples' 'bananas' 'dislike' 'hate' 'like' 'oranges' 'they'
 'we']
[[0 0 1 0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0 1 0 1]
 [1 0 0 1 0 0 0 0 0 0]
 [0 1 1 0 0 0 2 1 0 1]
 [0 0 0 1 1 0 0 0 1 0]]


In [224]:
reviews_test_example = ["They like bananas",
                   "We hate oranges bananas and apples",
                   "We love bananas"] #New word!

X_test_example = vectorizer.transform(reviews_test_example)

print(X_test_example.todense())

[[0 0 0 1 0 0 1 0 1 0]
 [0 1 1 1 0 1 0 1 0 1]
 [0 0 0 1 0 0 0 0 0 1]]


We should acknowledge few facts. Firstly, CountVectorizer does not take order into account. Secondly, it ignores one-letter words (this can be changed during initialization). Finally, for test values, CountVectorizer ignores words which are not in it's dictionary.

## Exercise 2
a) Split dataset into training and test sets.     
b) Transform reviews into vectors using CountVectorizer.

In [225]:
#a)
from sklearn.model_selection import train_test_split
#spliting reviews and rating into train and test sets
reviews_train, reviews_test = train_test_split(
    baby_df['review'], test_size=0.2, random_state=42
)
y_train, y_test = train_test_split(baby_df['rating'], test_size=0.2,random_state=42 )


In [226]:
#b)
#using countVectorizer
vectorizer = CountVectorizer()
#using data to create a vocabulary and generate matrix
X_train_vectors = vectorizer.fit_transform(reviews_train)
X_test_vectors = vectorizer.transform(reviews_test)
#printing 10 words from learned vocabulary to check if code is correct
print(list(vectorizer.vocabulary_.keys())[:10])




['you', 'cannot', 'imagine', 'how', 'much', 'money', 'youll', 'save', 'and', 'it']


## Exercise 3
a) Train LogisticRegression model on training data (reviews processed with CountVectorizer, ratings as they were).   
b) Print 10 most positive and 10 most negative words.

In [ ]:
#a)
from sklearn.metrics import accuracy_score
#creating LogisticRegression model and fitting data
model = LogisticRegression()
model.fit(X_train_vectors, y_train)


c:\Users\Dell Latitude E5550\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
#b)
print(model.coef_) #this is an array of coefficients values for each word [0] - positive and [1] - negative
# then we are taking each coefficiency value and assing it to it's word from vocabulary
coefficients = model.coef_[0] 
vocabulary = vectorizer.get_feature_names_out()
# creating a list with words and assigned to them coefficients
word_coef = list(zip(vocabulary, coefficients)) 
# sorting to find most positive and most negative words
sorted_words = sorted(word_coef, key=lambda x: x[1], reverse=True)

most_positive = sorted_words[:10]
most_negative = sorted_words[-10:]
#positive words are those that influence the model the most to tell that this specific review is positive
print("10 most positive words:")
for word, coef in most_positive:
    print(f"{word}: {coef:.4f}")
#negative words are those that influence the model the most to tell that this specific review is negative
print("\n10 most negative words:")
for word, coef in most_negative:
    print(f"{word}: {coef:.4f}")

#hint: model.coef_, vectorizer.get_feature_names()

[[ 0.00129108  0.0041022   0.01298436 ...  0.00145851  0.00087492
  -0.00030991]]
10 najbardziej pozytywnych słów:
excellent: 2.4087
awesome: 2.2799
worry: 2.2030
amazing: 2.1287
pleased: 2.1259
satisfied: 2.0490
perfect: 2.0022
complaint: 1.9495
glad: 1.9293
loves: 1.8213

10 najbardziej negatywnych słów:
disappointment: -2.2972
returning: -2.3408
disappointed: -2.3514
poor: -2.5110
terrible: -2.5589
concept: -2.5996
poorly: -2.9549
useless: -3.0046
disappointing: -3.2313
worst: -3.3015


## Exercise 4
a) Predict the sentiment of test data reviews.   
b) Predict the sentiment of test data reviews in terms of probability.   
c) Find five most positive and most negative reviews.   
d) Calculate the accuracy of predictions.

In [229]:
#a)
y_pred = model.predict(X_test_vectors)



In [230]:
#b)
y_pred_proba = model.predict_proba(X_test_vectors)

#hint: model.predict_proba()

In [ ]:
#c)
#this is an array that contains of proabilieties for each class
positive_indices = np.argsort(y_pred_proba[:, 1])[-5:]
negative_indices = np.argsort(y_pred_proba[:, 0])[-5:]
#extracting the reviews from their indices using .iloc
most_positive_reviews = reviews_test.iloc[positive_indices]
most_negative_reviews = reviews_test.iloc[negative_indices]
#the 5 reviews with the highest positive/negative probabilities
print("\n5 most positive reviews:")
for i, review in enumerate(most_positive_reviews):
    print(f"Review {i + 1}: {review}")
print("\n5 most negative reviews:")
for i, review in enumerate(most_negative_reviews):
    print(f"Review {i + 1}: {review}")

#hint: use the results of b)


5 najbardziej pozytywnych recenzji:
Recenzja 1: I started wearing the Babyplus when I was 18 weeks pregnant My husband was skeptical but I figured it couldnt hurt I wore it for an hour every morning and an hour every night  I often wore it in the car since I had a 1 hour commute each way  I never found the sounds annoying  My son was born alert and calm he didnt even cry  He was such a good easy baby he really only cried if he was gassy or very hungry  Hes always had a really good disposition and people even strangers often comment about how sweet happy and easy going he is  Hes reached all his milestones early  He rolled over sat up and crawled early  He took his first steps one week shy of becoming 10 months old and was fully walking at 11 monthsIve also taught him sign language  He picked it up very quickly and he also started speaking at an early age  His language amazes me  He turned 16 months old a few days ago and can sign over 50 signs can speak over 120 words and even say 12 

In [ ]:
#d)
#calculating accuracy
y_pred = model.predict(X_test_vectors)
accuracy1 = accuracy_score(y_test, y_pred)

print(f'Accuracy: {accuracy1:.4f}')
#as we can see accuracy is very high

Accuracy: 0.9350


## Exercise 5
In this exercise we will limit the dictionary of CountVectorizer to the set of significant words, defined below.


a) Redo exercises 2-5 using limited dictionary.   
b) Check the impact of all the words from the dictionary.   
c) Compare accuracy of predictions and the time of evaluation.

In [233]:
significant_words = ['love','great','easy','old','little','perfect','loves','well','able','car','broke','less','even','waste','disappointed','work','product','money','would','return']

In [234]:
#a)
model2 = LogisticRegression()

vectorizer = CountVectorizer(vocabulary=significant_words)

X_train_vectors = vectorizer.fit_transform(reviews_train)
X_test_vectors = vectorizer.transform(reviews_test)

model2.fit(X_train_vectors, y_train)
y_pred_limited = model2.predict(X_test_vectors)
coefficients = model2.coef_[0]
vocabulary = vectorizer.get_feature_names_out()




In [ ]:
#b)
word_coef = list(zip(vocabulary, coefficients)) 
sorted_words = sorted(word_coef, key=lambda x: x[1], reverse=True)

most_positive = sorted_words[:10]
most_negative = sorted_words[-10:]

print("10 most positive words:")
for word, coef in most_positive:
    print(f"{word}: {coef:.4f}")

print("\n10 most negative words:")
for word, coef in most_negative:
    print(f"{word}: {coef:.4f}")


10 najbardziej pozytywnych słów:
loves: 1.6850
perfect: 1.5151
love: 1.3590
easy: 1.1932
great: 0.9309
little: 0.5024
well: 0.4962
able: 0.1933
car: 0.0745
old: 0.0734

10 najbardziej negatywnych słów:
less: -0.2016
product: -0.3137
would: -0.3422
even: -0.4897
work: -0.6356
money: -0.9464
broke: -1.6806
waste: -1.9796
return: -2.0928
disappointed: -2.3988


In [ ]:
#c)
#at the end we are compering models accuracies and time of evaluation
#with more features the model can better capture nuances and subtle patterns in the text
#leading to improved classification
#time of evaluation is expectedly longer when a vocabulary is richer
y_pred = model2.predict(X_test_vectors)
accuracy2 = accuracy_score(y_test, y_pred)
print("Accuracy of full dictionary")
print(f'Accuracy: {accuracy1:.4f}')
print("Accuracy of limited dictionary ")
print(f'Accuracy: {accuracy2:.4f}')
#hint: %time, %timeit
print("Time of evaluation for full dictionary ")
%time model.fit(X_train_vectors, y_train)
%timeit model.predict(X_test_vectors)
print("Time of evaluation for limited dictionary ")
%time model2.fit(X_train_vectors, y_train)
%timeit model2.predict(X_test_vectors)




Accuracy of full dictionary
Accuracy: 0.9350
Accuracy of limited dictionary 
Accuracy: 0.8690
Time of evaluation for full dictionary 
CPU times: total: 219 ms
Wall time: 213 ms
1.02 ms ± 24.7 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
Time of evaluation for limited dictionary 
CPU times: total: 188 ms
Wall time: 205 ms
1.26 ms ± 332 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
